In [1]:
library(tidyverse)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
dset_idx <- 1

# Load and filter survival data

In [4]:
event_code <- list("Alive" = 0, "Dead" = 1)
covariate_cols_no_figo <- c("age_at_diagnosis", "race", "ethnicity")
covariate_cols <- c("figo_stage", covariate_cols_no_figo)
dep_cols <- c("vital_status", "survival_time")

In [5]:
survival_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv")
survival_df <- load_survival_df(survival_path, event_code)

Parsed with column specification:
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)


In [6]:
filtered_survival_df <- survival_df %>%
    dplyr::select(sample_name, vital_status, survival_time) %>%
    dplyr::filter(vital_status == event_code$Dead, rowSums(is.na(.)) == 0)
    
nrow(filtered_survival_df)
head(filtered_survival_df)

[1] 66

sample_name,vital_status,survival_time
<chr>,<dbl>,<dbl>
TCGA-C5-A1BF-01B-11R-A13Y-07,1,570
TCGA-C5-A8YT-01A-11R-A37O-07,1,633
TCGA-C5-A2LZ-01A-11R-A213-07,1,3046
TCGA-VS-A9V1-01A-11R-A42T-07,1,157
TCGA-C5-A1BE-01B-11R-A13Y-07,1,2094
TCGA-C5-A8XH-01A-11R-A37O-07,1,1394


# Load normalized matrisome count data

In [7]:
norm_matrisome_counts_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/norm_matrisome_counts.tsv")
norm_survival_counts_df <- read_tsv(norm_matrisome_counts_path) %>%
    dplyr::select(c("geneID", filtered_survival_df$sample_name))

Parsed with column specification:
cols(
  .default = col_double(),
  geneID = col_character()
)
See spec(...) for full column specifications.


In [8]:
norm_survival_counts_t_df <- norm_survival_counts_df %>%
    column_to_rownames(var = "geneID") %>%
    t() %>%
    as_tibble(rownames = "sample_name") %>%
    inner_join(filtered_survival_df, by = "sample_name") %>%
    dplyr::select(sample_name, survival_time, everything(), -vital_status)

In [9]:
nrow(norm_survival_counts_t_df)
head(norm_survival_counts_t_df)

[1] 66

sample_name,survival_time,PGF,TIMP4,C1QTNF6,TNC,PRL,OGN,C1QL3,FGB,⋯,PIK3IP1,C1QTNF2,PCSK5,ANXA1,HGF,VWA2,FGF3,POSTN,NTF3,S100A6
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TCGA-C5-A1BF-01B-11R-A13Y-07,570,10.800637,6.228003,11.669331,13.00293,5.063964,4.869744,5.063964,8.834522,⋯,9.013453,8.190325,9.503647,14.07799,6.569726,7.315604,4.602649,12.062300,5.649441,16.55841
TCGA-C5-A8YT-01A-11R-A37O-07,633,7.830611,5.733875,12.445548,13.76547,5.455125,13.049104,5.146455,5.074289,⋯,9.453187,6.398956,12.288955,13.39633,10.228758,8.542025,4.602649,11.765396,5.318924,13.55632
TCGA-C5-A2LZ-01A-11R-A213-07,3046,10.135550,4.865349,8.566221,10.84231,4.865349,5.241781,6.669045,4.602649,⋯,11.492135,7.426145,9.480199,15.92715,6.010764,7.405245,4.602649,7.506731,5.423416,14.73802
TCGA-VS-A9V1-01A-11R-A42T-07,157,6.893100,6.334201,10.871571,12.21604,4.602649,5.153289,5.153289,6.749410,⋯,9.319180,7.444380,8.707254,12.23439,6.011689,10.391491,4.602649,7.767144,5.376786,16.71254
TCGA-C5-A1BE-01B-11R-A13Y-07,2094,10.642039,5.348449,8.945220,13.41922,4.602649,5.867905,5.646251,4.602649,⋯,11.696884,6.384980,9.774029,15.38194,6.205261,7.163053,4.957257,10.113788,5.213815,15.56497
TCGA-C5-A8XH-01A-11R-A37O-07,1394,9.633752,5.908552,11.672191,13.86377,4.602649,5.339887,5.702581,5.389600,⋯,11.148165,7.526140,8.561116,14.40442,6.462928,6.102570,4.602649,9.104547,5.088257,15.19735


In [11]:
filtered_survival_counts_t_df <- filter_outliers_IQR(norm_survival_counts_t_df, "survival_time", coef = 1.5)

In [12]:
cor_test_df <- colwise_cor_test(
    filtered_survival_counts_t_df,
    colnames(filtered_survival_counts_t_df)[-c(1:2)],
    "survival_time",
    v = "geneID"
)

Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”Warning message in cor(x, y):
“the standard deviation is zero”

In [13]:
dim(cor_test_df)
head(cor_test_df)

[1] 1008    4

geneID,cor,pval,n
<chr>,<dbl>,<dbl>,<int>
PGF,-0.239511697,0.06301199,61
TIMP4,-0.060178185,0.64501303,61
C1QTNF6,-0.275997697,0.03131662,61
TNC,0.009778688,0.94037723,61
PRL,-0.126888957,0.32981912,61
OGN,0.179234460,0.16693270,61


In [14]:
nrow(cor_test_df %>%
    dplyr::filter(pval < 0.05))

[1] 108

In [ ]:
write_tsv(cor_test_df, paste0(dirs$analysis_dir, "/", unified_dsets[dset_idx], "_cor_results_outliers_removed.tsv"))